# 04 - Complete Calibration Workflows

This notebook demonstrates complete calibration workflows for quantum devices in LeeQ.

## Learning Objectives
- Understand automated calibration procedures
- Learn calibration optimization strategies
- Practice with measurement calibration
- Explore calibration data management

## Prerequisites
- Complete [03_multi_qubit.ipynb](03_multi_qubit.ipynb)
- Understand quantum device characterization

## Setup and Configuration

In [ ]:
# Import required modules
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import json

# Import LeeQ modules
from leeq.chronicle import Chronicle
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import calibration experiments
from leeq.experiments.builtin.basic.calibrations import *
from leeq.experiments.builtin.basic.characterizations import *

# Start Chronicle logging
Chronicle().start_log()
print("Chronicle logging started for calibration session")

## Initialize Quantum System

Set up a virtual quantum system for calibration demonstrations.

In [ ]:
# Create experiment manager
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon with realistic drift
virtual_qubit = VirtualTransmon(
    name="CalibQubit",
    qubit_frequency=5000.0,  # Initial frequency (will drift)
    anharmonicity=-200,
    t1=75,
    t2=40,
    readout_frequency=9500.0,
    quiescent_state_distribution=np.asarray([0.85, 0.12, 0.02, 0.01])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='CalibrationSetup',
    virtual_qubits={1: virtual_qubit}
)

manager.register_setup(setup)

# Configure shot parameters
setup.status().set_param("Shot_Number", 1000)
setup.status().set_param("Shot_Period", 200)

print("Calibration system initialized")

In [ ]:
# Initialize TransmonElement with initial guess parameters
initial_config = {
    'hrid': 'Q_CAL',
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5000.0,  # Initial guess
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.5,  # Initial guess
            'phase': 0.,
            'width': 0.05,
            'alpha': 500,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 4800.0,  # Initial guess for f12
            'channel': 1,
            'shape': 'blackman_drag',
            'amp': 0.07,
            'phase': 0.,
            'width': 0.025,
            'alpha': 400,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9500.0,  # Initial guess
            'channel': 0,
            'shape': 'square',
            'amp': 0.1,  # Initial guess
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        }
    }
}

qubit = TransmonElement(name='Q_CAL', parameters=initial_config)
print("Qubit element created with initial parameters")
qubit.print_config_info()

## Automated Calibration Procedures

### Calibration Workflow Overview

A complete calibration workflow typically follows this sequence:

1. **Resonator Spectroscopy** - Find readout frequency
2. **Qubit Spectroscopy** - Find qubit frequency
3. **Rabi Oscillations** - Calibrate pulse amplitude
4. **Ramsey Fringes** - Fine-tune frequency
5. **Measurement Calibration** - Optimize readout
6. **DRAG Calibration** - Minimize leakage
7. **T1/T2 Characterization** - Measure coherence

In [ ]:
class AutomatedCalibrationWorkflow:
    """
    Automated calibration workflow manager.
    """
    
    def __init__(self, qubit_element):
        self.qubit = qubit_element
        self.calibration_log = []
        self.parameters = {}
        
    def log_calibration(self, step_name, result, parameters):
        """Log calibration step results."""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'step': step_name,
            'result': result,
            'parameters': parameters
        }
        self.calibration_log.append(entry)
        Chronicle().log_event("calibration_step", entry)
        
    def run_resonator_spectroscopy(self):
        """Step 1: Find resonator frequency."""
        print("\n" + "="*60)
        print("STEP 1: Resonator Spectroscopy")
        print("="*60)
        
        # Define search range around expected frequency
        center_freq = self.qubit.get_measurement_prim_intlist(0)['freq']
        scan_range = 20  # MHz
        
        params = {
            'start': center_freq - scan_range,
            'stop': center_freq + scan_range,
            'step': 0.5,
            'amp': 0.05,
            'width': 2.0
        }
        
        print(f"Scanning resonator: {params['start']:.1f} - {params['stop']:.1f} MHz")
        print(f"Step size: {params['step']} MHz")
        
        # Simulate finding resonator (would be actual experiment)
        found_freq = center_freq + np.random.normal(0, 0.5)  # Simulated drift
        
        # Update qubit parameters
        self.qubit.update_measurement_prim(0, {'freq': found_freq})
        
        result = f"Found at {found_freq:.2f} MHz"
        self.log_calibration('resonator_spectroscopy', result, params)
        
        print(f"✓ Resonator frequency: {found_freq:.2f} MHz")
        return found_freq
        
    def run_qubit_spectroscopy(self):
        """Step 2: Find qubit frequency."""
        print("\n" + "="*60)
        print("STEP 2: Qubit Spectroscopy")
        print("="*60)
        
        # Define search range
        center_freq = self.qubit.get_c1('f01')['freq']
        scan_range = 50  # MHz
        
        params = {
            'start': center_freq - scan_range,
            'stop': center_freq + scan_range,
            'step': 1.0,
            'pulse_amp': 0.1,
            'pulse_width': 10.0  # Long weak pulse
        }
        
        print(f"Scanning qubit: {params['start']:.1f} - {params['stop']:.1f} MHz")
        
        # Simulate finding qubit (would be actual experiment)
        found_freq = center_freq + np.random.normal(0, 2)  # Simulated drift
        
        # Update qubit parameters
        self.qubit.update_lpb_collection('f01', {'freq': found_freq})
        
        result = f"Found at {found_freq:.2f} MHz"
        self.log_calibration('qubit_spectroscopy', result, params)
        
        print(f"✓ Qubit frequency: {found_freq:.2f} MHz")
        return found_freq
        
    def run_rabi_calibration(self):
        """Step 3: Calibrate pulse amplitude."""
        print("\n" + "="*60)
        print("STEP 3: Rabi Amplitude Calibration")
        print("="*60)
        
        params = {
            'amp_start': 0.0,
            'amp_stop': 1.0,
            'amp_step': 0.02,
            'pulse_width': 0.05
        }
        
        print(f"Amplitude scan: {params['amp_start']} - {params['amp_stop']}")
        
        # Simulate Rabi oscillation and find pi pulse
        ideal_amp = 0.55
        found_amp = ideal_amp + np.random.normal(0, 0.01)
        
        # Update qubit parameters
        self.qubit.update_lpb_collection('f01', {'amp': found_amp})
        
        result = f"π pulse amplitude: {found_amp:.4f}"
        self.log_calibration('rabi_calibration', result, params)
        
        print(f"✓ π pulse amplitude: {found_amp:.4f}")
        print(f"✓ π/2 pulse amplitude: {found_amp/2:.4f}")
        return found_amp
        
    def run_ramsey_calibration(self):
        """Step 4: Fine-tune frequency with Ramsey."""
        print("\n" + "="*60)
        print("STEP 4: Ramsey Frequency Calibration")
        print("="*60)
        
        params = {
            'delay_start': 0,
            'delay_stop': 5.0,  # microseconds
            'delay_step': 0.05,
            'detuning': 0.5  # MHz artificial detuning
        }
        
        print(f"Ramsey delay scan: {params['delay_start']} - {params['delay_stop']} μs")
        
        # Simulate finding frequency offset
        freq_offset = np.random.normal(0, 0.1)  # MHz
        current_freq = self.qubit.get_c1('f01')['freq']
        corrected_freq = current_freq + freq_offset
        
        # Update frequency
        self.qubit.update_lpb_collection('f01', {'freq': corrected_freq})
        
        result = f"Frequency offset: {freq_offset:.3f} MHz"
        self.log_calibration('ramsey_calibration', result, params)
        
        print(f"✓ Frequency correction: {freq_offset:+.3f} MHz")
        print(f"✓ Final frequency: {corrected_freq:.3f} MHz")
        return corrected_freq
        
    def run_full_calibration(self):
        """Run complete calibration workflow."""
        print("\n" + "#"*60)
        print("#" + " "*18 + "AUTOMATED CALIBRATION" + " "*19 + "#")
        print("#"*60)
        
        start_time = datetime.now()
        
        # Run calibration sequence
        res_freq = self.run_resonator_spectroscopy()
        qubit_freq = self.run_qubit_spectroscopy()
        pi_amp = self.run_rabi_calibration()
        final_freq = self.run_ramsey_calibration()
        
        # Summary
        elapsed = (datetime.now() - start_time).total_seconds()
        
        print("\n" + "#"*60)
        print("#" + " "*18 + "CALIBRATION COMPLETE" + " "*20 + "#")
        print("#"*60)
        print(f"\nTotal time: {elapsed:.1f} seconds")
        print(f"Steps completed: {len(self.calibration_log)}")
        
        return {
            'resonator_freq': res_freq,
            'qubit_freq': final_freq,
            'pi_amplitude': pi_amp,
            'calibration_log': self.calibration_log
        }

# Run automated calibration
calibrator = AutomatedCalibrationWorkflow(qubit)
calibration_results = calibrator.run_full_calibration()

## Measurement Calibration

Optimize measurement parameters for maximum state discrimination fidelity.

In [ ]:
# Import measurement calibration modules
from leeq.experiments.builtin.basic.calibrations.state_discrimination import (
    MeasurementCalibrationMultilevelGMM,
    MeasurementOptimization
)

def optimize_measurement_parameters(qubit_element):
    """
    Optimize measurement parameters for state discrimination.
    """
    print("\n" + "="*60)
    print("Measurement Parameter Optimization")
    print("="*60)
    
    # Define optimization ranges
    optimization_params = {
        'frequency_range': np.linspace(-5, 5, 11),  # MHz relative to current
        'amplitude_range': np.linspace(0.05, 0.3, 11),
        'phase_range': np.linspace(-np.pi, np.pi, 11),
        'duration_range': np.linspace(0.5, 3.0, 11)  # microseconds
    }
    
    # Simulate optimization results
    current_freq = qubit_element.get_measurement_prim_intlist(0)['freq']
    
    # Simulated optimal parameters
    optimal_params = {
        'freq': current_freq + np.random.normal(0, 0.5),
        'amp': 0.15 + np.random.normal(0, 0.02),
        'phase': np.random.normal(0, 0.1),
        'width': 1.5 + np.random.normal(0, 0.1)
    }
    
    # Calculate discrimination metrics
    snr = 8.5 + np.random.normal(0, 0.5)  # Signal-to-noise ratio
    fidelity = 0.98 + np.random.normal(0, 0.005)  # State discrimination fidelity
    
    print("\nOptimization Results:")
    print(f"  Frequency: {optimal_params['freq']:.2f} MHz")
    print(f"  Amplitude: {optimal_params['amp']:.3f}")
    print(f"  Phase: {optimal_params['phase']:.3f} rad")
    print(f"  Duration: {optimal_params['width']:.2f} μs")
    print(f"\nPerformance Metrics:")
    print(f"  SNR: {snr:.1f}")
    print(f"  Fidelity: {fidelity:.1%}")
    
    return optimal_params, {'snr': snr, 'fidelity': fidelity}

# Run measurement optimization
meas_params, meas_metrics = optimize_measurement_parameters(qubit)

In [ ]:
# Visualize measurement distributions
def visualize_measurement_distributions():
    """
    Visualize IQ distributions for different qubit states.
    """
    # Generate simulated IQ data
    n_shots = 1000
    
    # State |0⟩ distribution
    i0 = np.random.normal(1.0, 0.15, n_shots)
    q0 = np.random.normal(0.0, 0.15, n_shots)
    
    # State |1⟩ distribution
    i1 = np.random.normal(-0.8, 0.15, n_shots)
    q1 = np.random.normal(0.2, 0.15, n_shots)
    
    # State |2⟩ distribution (leakage)
    i2 = np.random.normal(0.0, 0.2, n_shots//10)
    q2 = np.random.normal(1.2, 0.2, n_shots//10)
    
    # Create scatter plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=i0, y=q0,
        mode='markers',
        marker=dict(size=3, color='blue', opacity=0.5),
        name='|0⟩ state'
    ))
    
    fig.add_trace(go.Scatter(
        x=i1, y=q1,
        mode='markers',
        marker=dict(size=3, color='red', opacity=0.5),
        name='|1⟩ state'
    ))
    
    fig.add_trace(go.Scatter(
        x=i2, y=q2,
        mode='markers',
        marker=dict(size=3, color='green', opacity=0.5),
        name='|2⟩ state (leakage)'
    ))
    
    # Add decision boundaries
    theta = np.linspace(0, 2*np.pi, 100)
    for center, radius, name in [
        ([1.0, 0.0], 0.5, 'Decision boundary |0⟩'),
        ([-0.8, 0.2], 0.5, 'Decision boundary |1⟩'),
    ]:
        x_circle = center[0] + radius * np.cos(theta)
        y_circle = center[1] + radius * np.sin(theta)
        fig.add_trace(go.Scatter(
            x=x_circle, y=y_circle,
            mode='lines',
            line=dict(dash='dash', color='gray'),
            showlegend=False
        ))
    
    fig.update_layout(
        title='IQ Measurement Distributions',
        xaxis_title='I Quadrature',
        yaxis_title='Q Quadrature',
        width=700,
        height=600,
        showlegend=True
    )
    
    fig.show()
    
    # Calculate and display separation metrics
    separation_01 = np.sqrt((1.0 - (-0.8))**2 + (0.0 - 0.2)**2)
    print(f"\nState Separation Metrics:")
    print(f"  |0⟩-|1⟩ separation: {separation_01:.2f} (IQ units)")
    print(f"  SNR: {separation_01/0.15:.1f}")
    print(f"  Theoretical fidelity: {0.5 * (1 + np.erf(separation_01/0.15/np.sqrt(2))):.1%}")

visualize_measurement_distributions()

## DRAG Calibration

Calibrate DRAG (Derivative Removal by Adiabatic Gate) parameters to minimize leakage.

In [ ]:
from leeq.experiments.builtin.basic.calibrations.drag import (
    CrossAllXYDragMultiRunSingleQubitMultilevel
)

def calibrate_drag_parameters(qubit_element):
    """
    Calibrate DRAG parameters to minimize leakage to |2⟩ state.
    """
    print("\n" + "="*60)
    print("DRAG Parameter Calibration")
    print("="*60)
    
    # DRAG calibration scans the alpha parameter
    alpha_range = np.linspace(-1000, 1000, 21)
    
    # Simulate DRAG calibration results
    leakage_rates = []
    phase_errors = []
    
    for alpha in alpha_range:
        # Simulate leakage and phase error vs alpha
        leakage = 0.01 * (1 + 0.5 * (alpha/500)**2) + np.random.normal(0, 0.001)
        phase = 0.05 * alpha/500 + np.random.normal(0, 0.005)
        leakage_rates.append(leakage)
        phase_errors.append(phase)
    
    # Find optimal alpha (minimum leakage)
    min_idx = np.argmin(leakage_rates)
    optimal_alpha = alpha_range[min_idx]
    
    # Create visualization
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=alpha_range,
        y=np.array(leakage_rates) * 100,
        mode='lines+markers',
        name='Leakage Rate',
        yaxis='y'
    ))
    
    fig.add_trace(go.Scatter(
        x=alpha_range,
        y=phase_errors,
        mode='lines+markers',
        name='Phase Error',
        yaxis='y2'
    ))
    
    # Mark optimal point
    fig.add_trace(go.Scatter(
        x=[optimal_alpha],
        y=[leakage_rates[min_idx] * 100],
        mode='markers',
        marker=dict(size=12, color='red', symbol='star'),
        name='Optimal',
        showlegend=True
    ))
    
    fig.update_layout(
        title='DRAG Calibration Results',
        xaxis_title='DRAG α Parameter',
        yaxis=dict(
            title='Leakage Rate (%)',
            side='left'
        ),
        yaxis2=dict(
            title='Phase Error (rad)',
            overlaying='y',
            side='right'
        ),
        width=800,
        height=500
    )
    
    fig.show()
    
    print(f"\nOptimal DRAG Parameters:")
    print(f"  α = {optimal_alpha:.1f}")
    print(f"  Leakage rate: {leakage_rates[min_idx]:.1%}")
    print(f"  Phase error: {phase_errors[min_idx]:.3f} rad")
    
    # Update qubit parameters
    qubit_element.update_lpb_collection('f01', {'alpha': optimal_alpha})
    
    return optimal_alpha

optimal_drag = calibrate_drag_parameters(qubit)

## Calibration Data Management

Save, load, and track calibration data using Chronicle.

In [ ]:
class CalibrationDataManager:
    """
    Manage calibration data storage and retrieval.
    """
    
    def __init__(self, qubit_name):
        self.qubit_name = qubit_name
        self.calibration_history = []
        
    def save_calibration(self, calibration_data):
        """
        Save calibration data to Chronicle.
        """
        timestamp = datetime.now()
        
        # Create calibration record
        record = {
            'timestamp': timestamp.isoformat(),
            'qubit': self.qubit_name,
            'parameters': calibration_data,
            'metadata': {
                'temperature': '10mK',  # Example metadata
                'operator': 'AutoCalibration',
                'version': '1.0'
            }
        }
        
        # Log to Chronicle
        Chronicle().log_event('calibration_save', record)
        
        # Add to history
        self.calibration_history.append(record)
        
        print(f"\n✓ Calibration saved at {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        return record
        
    def load_latest_calibration(self):
        """
        Load the most recent calibration.
        """
        if not self.calibration_history:
            print("No calibration history available")
            return None
            
        latest = self.calibration_history[-1]
        print(f"\n✓ Loaded calibration from {latest['timestamp']}")
        return latest['parameters']
        
    def compare_calibrations(self, cal1, cal2):
        """
        Compare two calibration sets.
        """
        print("\nCalibration Comparison:")
        print("="*40)
        
        # Compare key parameters
        params_to_compare = ['qubit_freq', 'pi_amplitude', 'resonator_freq']
        
        for param in params_to_compare:
            if param in cal1 and param in cal2:
                val1 = cal1[param]
                val2 = cal2[param]
                diff = val2 - val1 if isinstance(val1, (int, float)) else 'N/A'
                
                print(f"{param:20s}: {val1:10.4f} → {val2:10.4f} (Δ = {diff:+.4f})")
                
    def generate_calibration_report(self):
        """
        Generate a calibration report.
        """
        print("\n" + "="*60)
        print("CALIBRATION REPORT")
        print("="*60)
        
        if not self.calibration_history:
            print("No calibration data available")
            return
            
        latest = self.calibration_history[-1]
        
        print(f"\nQubit: {self.qubit_name}")
        print(f"Timestamp: {latest['timestamp']}")
        print(f"\nCalibrated Parameters:")
        
        for key, value in latest['parameters'].items():
            if isinstance(value, (int, float)):
                print(f"  {key:25s}: {value:.6f}")
            elif isinstance(value, dict):
                print(f"  {key:25s}: [complex data]")
            else:
                print(f"  {key:25s}: {value}")
                
        print(f"\nMetadata:")
        for key, value in latest['metadata'].items():
            print(f"  {key:25s}: {value}")

# Create data manager and save calibration
data_manager = CalibrationDataManager('Q_CAL')
saved_record = data_manager.save_calibration(calibration_results)

# Generate report
data_manager.generate_calibration_report()

## Calibration Drift Tracking

Monitor and visualize parameter drift over time.

In [ ]:
def simulate_calibration_drift():
    """
    Simulate and visualize calibration drift over time.
    """
    # Simulate multiple calibration runs over 24 hours
    n_calibrations = 24  # One per hour
    time_points = np.arange(n_calibrations)
    
    # Simulate drifting parameters
    base_freq = 5000.0
    base_amp = 0.55
    
    # Add realistic drift patterns
    freq_drift = base_freq + 0.5 * np.sin(2*np.pi*time_points/24) + \
                 np.cumsum(np.random.normal(0, 0.05, n_calibrations))
    
    amp_drift = base_amp + 0.01 * np.sin(2*np.pi*time_points/12) + \
                np.cumsum(np.random.normal(0, 0.001, n_calibrations))
    
    t1_values = 75 + 5*np.sin(2*np.pi*time_points/24) + \
                np.random.normal(0, 2, n_calibrations)
    
    # Create subplots
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('Qubit Frequency Drift', 'π Amplitude Drift', 'T1 Variation'),
        vertical_spacing=0.1
    )
    
    # Frequency drift
    fig.add_trace(
        go.Scatter(x=time_points, y=freq_drift, mode='lines+markers', name='Frequency'),
        row=1, col=1
    )
    fig.add_hline(y=base_freq, line_dash="dash", line_color="gray", row=1, col=1)
    
    # Amplitude drift
    fig.add_trace(
        go.Scatter(x=time_points, y=amp_drift, mode='lines+markers', name='Amplitude'),
        row=2, col=1
    )
    fig.add_hline(y=base_amp, line_dash="dash", line_color="gray", row=2, col=1)
    
    # T1 variation
    fig.add_trace(
        go.Scatter(x=time_points, y=t1_values, mode='lines+markers', name='T1'),
        row=3, col=1
    )
    fig.add_hline(y=75, line_dash="dash", line_color="gray", row=3, col=1)
    
    fig.update_xaxes(title_text="Time (hours)", row=3, col=1)
    fig.update_yaxes(title_text="Frequency (MHz)", row=1, col=1)
    fig.update_yaxes(title_text="Amplitude", row=2, col=1)
    fig.update_yaxes(title_text="T1 (μs)", row=3, col=1)
    
    fig.update_layout(
        height=800,
        title_text="Calibration Parameter Drift Over 24 Hours",
        showlegend=False
    )
    
    fig.show()
    
    # Calculate drift statistics
    print("\nDrift Statistics (24 hours):")
    print("="*40)
    print(f"Frequency:")
    print(f"  Mean: {np.mean(freq_drift):.2f} MHz")
    print(f"  Std: {np.std(freq_drift):.3f} MHz")
    print(f"  Max drift: {np.max(np.abs(freq_drift - base_freq)):.3f} MHz")
    
    print(f"\nπ Amplitude:")
    print(f"  Mean: {np.mean(amp_drift):.4f}")
    print(f"  Std: {np.std(amp_drift):.4f}")
    print(f"  Max drift: {np.max(np.abs(amp_drift - base_amp)):.4f}")
    
    print(f"\nT1:")
    print(f"  Mean: {np.mean(t1_values):.1f} μs")
    print(f"  Std: {np.std(t1_values):.1f} μs")
    print(f"  Min/Max: {np.min(t1_values):.1f} / {np.max(t1_values):.1f} μs")

simulate_calibration_drift()

## Daily Calibration Routine

Implement a daily calibration routine that runs automatically.

In [ ]:
class DailyCalibrationRoutine:
    """
    Automated daily calibration routine.
    """
    
    def __init__(self, qubits):
        self.qubits = qubits if isinstance(qubits, list) else [qubits]
        self.calibration_schedule = self.create_schedule()
        
    def create_schedule(self):
        """
        Create daily calibration schedule.
        """
        schedule = [
            {'time': '06:00', 'type': 'full', 'description': 'Morning full calibration'},
            {'time': '10:00', 'type': 'quick', 'description': 'Mid-morning touchup'},
            {'time': '14:00', 'type': 'quick', 'description': 'Afternoon touchup'},
            {'time': '18:00', 'type': 'full', 'description': 'Evening full calibration'},
            {'time': '22:00', 'type': 'quick', 'description': 'Night touchup'},
        ]
        return schedule
        
    def run_quick_calibration(self, qubit):
        """
        Quick calibration touchup (frequency and amplitude only).
        """
        steps = [
            "1. Ramsey frequency check",
            "2. Single-point Rabi amplitude check",
            "3. Measurement discrimination check"
        ]
        
        print("\n🔧 Quick Calibration:")
        for step in steps:
            print(f"  {step}")
            
        # Simulate quick calibration
        return {
            'duration': '5 minutes',
            'parameters_updated': ['frequency', 'amplitude'],
            'status': 'success'
        }
        
    def run_full_calibration(self, qubit):
        """
        Full calibration sequence.
        """
        steps = [
            "1. Resonator spectroscopy",
            "2. Qubit spectroscopy",
            "3. Rabi oscillations",
            "4. Ramsey fringes",
            "5. DRAG calibration",
            "6. Measurement optimization",
            "7. T1/T2 characterization"
        ]
        
        print("\n🔨 Full Calibration:")
        for step in steps:
            print(f"  {step}")
            
        # Simulate full calibration
        return {
            'duration': '30 minutes',
            'parameters_updated': 'all',
            'status': 'success'
        }
        
    def health_check(self, qubit):
        """
        Perform qubit health check.
        """
        metrics = {
            'T1': 75 + np.random.normal(0, 5),
            'T2_echo': 40 + np.random.normal(0, 3),
            'T2_ramsey': 35 + np.random.normal(0, 3),
            'gate_fidelity': 0.995 + np.random.normal(0, 0.002),
            'readout_fidelity': 0.98 + np.random.normal(0, 0.005)
        }
        
        # Check if any metric is below threshold
        issues = []
        if metrics['T1'] < 50:
            issues.append("T1 below threshold")
        if metrics['gate_fidelity'] < 0.99:
            issues.append("Gate fidelity below threshold")
            
        metrics['issues'] = issues
        metrics['health_status'] = 'healthy' if not issues else 'degraded'
        
        return metrics
        
    def display_schedule(self):
        """
        Display the daily calibration schedule.
        """
        print("\n" + "="*60)
        print("DAILY CALIBRATION SCHEDULE")
        print("="*60)
        
        for entry in self.calibration_schedule:
            icon = "🔨" if entry['type'] == 'full' else "🔧"
            print(f"{icon} {entry['time']:8s} - {entry['description']}")
            
    def run_calibration_at_time(self, time_str):
        """
        Run calibration for specific time.
        """
        # Find scheduled calibration
        scheduled = None
        for entry in self.calibration_schedule:
            if entry['time'] == time_str:
                scheduled = entry
                break
                
        if not scheduled:
            print(f"No calibration scheduled at {time_str}")
            return
            
        print(f"\n" + "="*60)
        print(f"Running: {scheduled['description']}")
        print(f"Time: {scheduled['time']}")
        print("="*60)
        
        results = []
        for qubit in self.qubits:
            print(f"\nCalibrating {qubit.name}...")
            
            # Run health check first
            health = self.health_check(qubit)
            print(f"  Health status: {health['health_status']}")
            
            # Run calibration
            if scheduled['type'] == 'full':
                result = self.run_full_calibration(qubit)
            else:
                result = self.run_quick_calibration(qubit)
                
            result['qubit'] = qubit.name
            result['health'] = health
            results.append(result)
            
            print(f"  Duration: {result['duration']}")
            print(f"  Status: {result['status']}")
            
        return results

# Create and run daily routine
daily_routine = DailyCalibrationRoutine(qubit)
daily_routine.display_schedule()

# Simulate morning calibration
morning_results = daily_routine.run_calibration_at_time('06:00')

## Calibration Quality Metrics

Track and visualize calibration quality over time.

In [ ]:
def create_calibration_dashboard():
    """
    Create a calibration quality dashboard.
    """
    from plotly.subplots import make_subplots
    
    # Generate sample data for multiple days
    days = 7
    calibrations_per_day = 5
    total_points = days * calibrations_per_day
    
    time_axis = np.arange(total_points)
    
    # Generate metrics
    gate_fidelities = 0.995 + 0.002 * np.sin(2*np.pi*time_axis/calibrations_per_day) + \
                      np.random.normal(0, 0.001, total_points)
    
    readout_fidelities = 0.98 + 0.01 * np.sin(2*np.pi*time_axis/calibrations_per_day/2) + \
                         np.random.normal(0, 0.002, total_points)
    
    calibration_times = 15 + 5 * np.sin(2*np.pi*time_axis/calibrations_per_day) + \
                       np.random.normal(0, 2, total_points)
    
    success_rate = np.random.choice([1, 1, 1, 1, 0], total_points)  # 80% success
    
    # Create dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Gate Fidelity Trend',
            'Readout Fidelity Trend',
            'Calibration Duration',
            'Success Rate'
        ),
        specs=[
            [{'type': 'scatter'}, {'type': 'scatter'}],
            [{'type': 'scatter'}, {'type': 'bar'}]
        ]
    )
    
    # Gate fidelity
    fig.add_trace(
        go.Scatter(x=time_axis, y=gate_fidelities, mode='lines+markers',
                  name='Gate Fidelity', line=dict(color='blue')),
        row=1, col=1
    )
    fig.add_hline(y=0.99, line_dash="dash", line_color="red",
                 annotation_text="Threshold", row=1, col=1)
    
    # Readout fidelity
    fig.add_trace(
        go.Scatter(x=time_axis, y=readout_fidelities, mode='lines+markers',
                  name='Readout Fidelity', line=dict(color='green')),
        row=1, col=2
    )
    fig.add_hline(y=0.97, line_dash="dash", line_color="red",
                 annotation_text="Threshold", row=1, col=2)
    
    # Calibration time
    fig.add_trace(
        go.Scatter(x=time_axis, y=calibration_times, mode='lines+markers',
                  name='Duration (min)', line=dict(color='orange')),
        row=2, col=1
    )
    
    # Success rate (daily)
    daily_success = []
    for d in range(days):
        start_idx = d * calibrations_per_day
        end_idx = (d + 1) * calibrations_per_day
        daily_success.append(np.mean(success_rate[start_idx:end_idx]) * 100)
    
    fig.add_trace(
        go.Bar(x=list(range(days)), y=daily_success,
              name='Success Rate (%)', marker_color='purple'),
        row=2, col=2
    )
    
    # Update layout
    fig.update_xaxes(title_text="Calibration Index", row=1, col=1)
    fig.update_xaxes(title_text="Calibration Index", row=1, col=2)
    fig.update_xaxes(title_text="Calibration Index", row=2, col=1)
    fig.update_xaxes(title_text="Day", row=2, col=2)
    
    fig.update_yaxes(title_text="Fidelity", row=1, col=1)
    fig.update_yaxes(title_text="Fidelity", row=1, col=2)
    fig.update_yaxes(title_text="Minutes", row=2, col=1)
    fig.update_yaxes(title_text="Success %", row=2, col=2)
    
    fig.update_layout(
        height=700,
        title_text="Calibration Quality Dashboard (7 Days)",
        showlegend=False
    )
    
    fig.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print("CALIBRATION QUALITY SUMMARY (7 Days)")
    print("="*60)
    print(f"\nGate Fidelity:")
    print(f"  Average: {np.mean(gate_fidelities):.4f}")
    print(f"  Min/Max: {np.min(gate_fidelities):.4f} / {np.max(gate_fidelities):.4f}")
    print(f"  Below threshold: {np.sum(gate_fidelities < 0.99)} times")
    
    print(f"\nReadout Fidelity:")
    print(f"  Average: {np.mean(readout_fidelities):.4f}")
    print(f"  Min/Max: {np.min(readout_fidelities):.4f} / {np.max(readout_fidelities):.4f}")
    print(f"  Below threshold: {np.sum(readout_fidelities < 0.97)} times")
    
    print(f"\nCalibration Performance:")
    print(f"  Average duration: {np.mean(calibration_times):.1f} minutes")
    print(f"  Success rate: {np.mean(success_rate)*100:.1f}%")
    print(f"  Total calibrations: {total_points}")

create_calibration_dashboard()

## Summary and Best Practices

### Key Takeaways

1. **Systematic Approach**: Follow a consistent calibration sequence
2. **Data Management**: Use Chronicle for comprehensive logging
3. **Automation**: Implement daily routines for consistent performance
4. **Monitoring**: Track drift and quality metrics continuously
5. **Optimization**: Regular refinement of calibration parameters

### Best Practices

#### Calibration Frequency
- **Full calibration**: 2-3 times daily
- **Quick touchup**: Every 2-4 hours
- **Health checks**: Before each experiment session

#### Parameter Priorities
1. **Critical**: Frequency, amplitude (drift fastest)
2. **Important**: DRAG, measurement parameters
3. **Periodic**: T1/T2, cross-talk characterization

#### Data Management
- Always save calibration data with Chronicle
- Include metadata (temperature, time, operator)
- Track parameter history for trend analysis
- Set up alerts for parameters outside bounds

#### Troubleshooting
- If fidelity drops suddenly, check for:
  - Temperature fluctuations
  - Electronic noise sources
  - Pulse distortions
- Maintain calibration logs for debugging
- Compare with historical data to identify anomalies

## Exercises

1. **Custom Calibration Sequence**: Design a calibration sequence optimized for your specific experiment needs
2. **Drift Prediction**: Implement a simple model to predict parameter drift
3. **Alert System**: Create an alert system for calibration parameters outside acceptable bounds
4. **Optimization Strategy**: Develop an adaptive calibration schedule based on drift patterns
5. **Multi-Qubit Calibration**: Extend the calibration workflow to handle multiple qubits in parallel

## Next Steps

Continue to [05_ai_integration.ipynb](05_ai_integration.ipynb) to learn about AI-assisted experiment generation, including:
- AI-powered experiment design
- Automated parameter optimization
- Intelligent calibration workflows
- LLM integration for experiment planning